In [1]:
import os
import os.path
from os import environ

In [2]:
# get environment variables section

# classification results from ccv
cnn_classification = environ.get("cnn_classification", None)

# file with the labels translation from cnn results (usually a *._words)
cnn_classnames = environ.get("cnn_classnames",None)

# first date from where to start ploting (in %Y-%m-%dT%H%%3A%M%%3A%S as saved in file)
min_date = environ.get("min_date", None)

# last date from where to start ploting (in %Y-%m-%dT%H%%3A%M%%3A%S as saved in file)
max_date = environ.get("max_date", None)

In [3]:
def get_merged_file(file_model_1, file_model_2, return_path):

    # check_var_people_background = all - people from file_model_1
    check_var_people_background = 0
    
    # if results file already exists, delete it.
    if os.path.exists(return_path):
        os.remove(return_path)
    
    # create file where to save all the selected
    ret_val = open(return_path, 'w')
    ret_val_lines = []
    
    # open person-people-background list of files
    m1 = open(file_model_1, 'r')
    m1_lines = m1.readlines()
    avoided_lines = 0
    for line in m1_lines:
        line_path, line_class, line_res = line.split(' ')
        new_class = -1

        # background
        if int(line_class) == 2:
            new_class = 5

        # people
        elif int(line_class) == 3:
            new_class = 4

        # person
        else:
            avoided_lines = avoided_lines + 1
            continue
        
        ret_val_lines.append(line_path+" "+str(new_class)+" "+line_res)

    check_var_people_background = len(ret_val_lines)

    # open poses list of files
    m2 = open(file_model_2, 'r')
    m2_lines = m2.readlines()
    
    for line in m2_lines:            
        ret_val_lines.append(line)
          
    # close files
    m1.close()
    m2.close()
    
    #write lines to file and exit
    ret_val.writelines(ret_val_lines)
    ret_val.close()
    
    # check numbers are ok
    people = (avoided_lines == len(m2_lines))
    total = ( check_var_people_background + len(m2_lines) == len(m1_lines) )
    
    return people and total

In [4]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import requests
from operator import itemgetter
from datetime import datetime, timedelta

requests.packages.urllib3.disable_warnings()
plotly.tools.set_credentials_file(username='pusiol', api_key='m6iurl7f89')

def to_unix_time(dt):
    epoch =  datetime.utcfromtimestamp(0)
    return (dt - epoch).total_seconds() * 1000

def get_timelapse_results(cnn_results, cnn_classnames, min_date, max_date, debug):
    data = get_data_and_classes(cnn_results, cnn_classnames)
    data = sorted(data, key=itemgetter(0))
    data = [(d,e) for d,e in data if d > min_date and d < max_date]
    
    data_lying = [d for d in data if d[1] == 'lying']
    data_sitting = [d for d in data if d[1] == 'sitting']
    data_standing = [d for d in data if d[1] == 'standing']
    data_people = [d for d in data if d[1] == 'people']
    data_background = [d for d in data if d[1] == 'background']
    
    '''
    if debug, we need to plot the frame position in array instead of the frame's date.
    '''
    if (debug):
        data_lying = [[data.index(d),''] for d in data if d[1] == 'lying']
        data_sitting = [[data.index(d),''] for d in data if d[1] == 'sitting']
        data_standing = [[data.index(d),''] for d in data if d[1] == 'standing']
        data_people = [[data.index(d),''] for d in data if d[1] == 'people']
        data_background = [[data.index(d),''] for d in data if d[1] == 'background']
    
    trace_init = go.Bar(
    x = [data[0][0]-timedelta(seconds=1)],
        y = ['time'],
        marker = dict(
        color = 'rgb(255, 255, 255)',
        )
    )
    
    trace_background = go.Bar(
        x = [d for d,_ in data_background],
        y = ['day 1'] * len(data_background),
        name = 'Background',
        #mode = 'markers',
        marker = dict(
            color = 'rgba(204, 0, 255, 1)',
        )
    )    
    
    trace_lying = go.Bar(
        x = [d for d,_ in data_lying],
        y = ['day 1'] * len(data_lying),
        name = 'Lying',
        #mode = 'markers',
        marker = dict(
            color = 'rgba(0, 0, 255, 1)',
        )
    ) 
    
    trace_sitting = go.Bar(
        x = [d for d,_ in data_sitting],
        y = ['day 1'] * len(data_sitting),
        name = 'Sitting',
        #mode = 'markers',
        marker = dict(
            color = 'rgba(51, 204, 51, 1)',
        )
    )
    
    trace_standing = go.Bar(
        x = [d for d,_ in data_standing],
        y = ['day 1'] * len(data_standing),
        name = 'Standing',
        #mode = 'markers',
        marker = dict(
            color = 'rgb(204, 102, 0)',
        )
    )
    
    trace_people = go.Bar(
        x = [d for d,_ in data_people],
        y = ['day 1'] * len(data_people),
        name = 'People',
        #mode = 'markers',
        marker = dict(
            color = 'rgb(255, 153, 0)',
        )
    )
    
    data = [trace_init ,trace_background, trace_lying, trace_standing, trace_sitting, trace_people]

    layout = dict(title = os.path.basename(cnn_results)+"_"+str(min_date)+"_to_"+str(max_date),
                  barmode='group',
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False),
                  height = 4,
                 )

    fig = dict(data=data, layout=layout)
    py.iplot(fig, filename=os.path.basename(cnn_results))

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [5]:
def get_data_and_classes(results_file, classes_file):

    data = []
    
    f = open(results_file, 'r')
    list_of_results = f.readlines()
    
    c = open(classes_file, 'r')
    list_of_classes = c.readlines()
    
    # remove \n of each class
    for cl in list_of_classes:
        cl = cl.rstrip()
    
    # for each file in the list of results, get pair datetime and classname and append to dates and classes accordingly.
    fixed_order = ["background", "lying", "sitting", "standing", "people"]
    for l in list_of_results:
        filename,class_id,_ = l.split(' ')
        data.append([get_date_from_filename(filename), list_of_classes[int(class_id)-1].rstrip()])
#        data.append([get_date_from_filename(filename), fixed_order.index(list_of_classes[int(class_id)-1].rstrip())+1])

    
    # close files
    f.close()
    c.close()
    
    # return values
    return data

def get_date_from_filename(filename):
    # name_of_video = "Users_thermaldata_unkown_2016-06-20_18%3A46%3A22.000000_2"
    name_of_video = os.path.basename(os.path.dirname(filename))

    # str_date_of_video = 2016-06-20T18%3A46%3A22
    str_date_of_video = "T".join(os.path.splitext(name_of_video)[0].split("_")[-2:])

    # date_of_video = datetime(2016-06-20 18:46:22)
    date_of_video = datetime.strptime(str_date_of_video,"%Y-%m-%dT%H%%3A%M%%3A%S")

    #convert to argentinian date
    date_of_video -= timedelta(hours=3)

    return date_of_video

In [12]:
# example for deep vgg model classification
from dateutil import *
cnn_classnames = "/media/Gui2/thermix/ARTraining/trained_models/thermix_38a/thermix_38a._words"
file_path_classif = "/media/Gui2/thermix/ARTraining/trained_models/thermix_38a/sarmiento_006_classify.txt"

# Uncomment to get first and last dates for both jupyter examples
data = get_data_and_classes(file_path_classif, cnn_classnames)
data = sorted(data, key=itemgetter(0))
first_date = data[0][0]
last_date = data[0][0] + timedelta(hours=24)

get_timelapse_results(cnn_results=file_path_classif, 
                      cnn_classnames=cnn_classnames, 
                      min_date=first_date, 
                      max_date=last_date,
                      debug=False
                     )


# # example for cascaded models with merge file generation
# file_path_1 = "/media/Gui2/thermix/ARTraining/trained_models/thermix_35a/full_day_fanny_classify.txt"
# file_path_2 = "/media/Gui2/thermix/ARTraining/trained_models/thermix_33a/full_day_fanny_poses_classify.txt"
# return_path = "/media/Gui2/thermix/ARTraining/merged_files/full_day_fanny_poses_merged.txt"
# numbers_match = get_merged_file(file_path_1, file_path_2, return_path)
# if not numbers_match:
#     assert False
# get_timelapse_results(cnn_results=return_path, 
#                      cnn_classnames=cnn_classnames, 
#                      min_date=first_date, 
#                      max_date=last_date
#                     )

# Bash environment variables run
'''
first_date = dateutil.parser.parse(min_date)
last_date = dateutil.parser.parse(max_date)
get_timelapse_results(cnn_results=cnn_classification,
                      cnn_classnames=cnn_classnames,
                      min_date=first_date,
                      max_date=last_date,
                      debug=False)

get_timelapse_results(cnn_results=cnn_classification,
                      cnn_classnames=cnn_classnames,
                      min_date=first_date,
                      max_date=last_date,
                      debug=True)
'''

The draw time for this plot will be slow for clients without much RAM.


/usr/local/lib/python2.7/dist-packages/plotly/plotly/plotly.py:1443: UserWarning:

Estimated Draw Time Slow



'\nfirst_date = dateutil.parser.parse(min_date)\nlast_date = dateutil.parser.parse(max_date)\nget_timelapse_results(cnn_results=cnn_classification,\n                      cnn_classnames=cnn_classnames,\n                      min_date=first_date,\n                      max_date=last_date,\n                      debug=False)\n\nget_timelapse_results(cnn_results=cnn_classification,\n                      cnn_classnames=cnn_classnames,\n                      min_date=first_date,\n                      max_date=last_date,\n                      debug=True)\n'